In [31]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import os
import json

## Project Discripition


In [32]:
# Pathway to use later to export CSV for cleaning.
covidTrackingDataPath = "/resources/daily_covid_tracking.csv"

# Country input statement to search for api
# country= input("Type a country you wish to serach for: ")

# URL for the API call
covidTrackingDataPath = f"https://api.covidtracking.com/v1/states/daily.json"

# Calling API and printing response.
response = requests.get(covidTrackingDataPath).json()
#print(json.dumps(response, indent=4, sort_keys=True))

In [33]:
response

[{'date': 20200923,
  'state': 'AK',
  'positive': 7941,
  'negative': 425257,
  'pending': None,
  'totalTestResults': 433198,
  'hospitalizedCurrently': 43,
  'hospitalizedCumulative': None,
  'inIcuCurrently': None,
  'inIcuCumulative': None,
  'onVentilatorCurrently': 14,
  'onVentilatorCumulative': None,
  'recovered': 2731,
  'dataQualityGrade': 'A',
  'lastUpdateEt': '9/23/2020 03:59',
  'dateModified': '2020-09-23T03:59:00Z',
  'checkTimeEt': '09/22 23:59',
  'death': 45,
  'hospitalized': None,
  'dateChecked': '2020-09-23T03:59:00Z',
  'totalTestsViral': 433198,
  'positiveTestsViral': 7180,
  'negativeTestsViral': 425736,
  'positiveCasesViral': 7941,
  'deathConfirmed': 45,
  'deathProbable': None,
  'totalTestEncountersViral': None,
  'totalTestsPeopleViral': None,
  'totalTestsAntibody': None,
  'positiveTestsAntibody': None,
  'negativeTestsAntibody': None,
  'totalTestsPeopleAntibody': None,
  'positiveTestsPeopleAntibody': None,
  'negativeTestsPeopleAntibody': None,
 

In [47]:
covidTrackingDF = pd.DataFrame(response)
covidTrackingDF

,date,state,positive,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200923,AK,7941.0,425257.0,NaN,433198.0,43.0,NaN,NaN,NaN,...,433198,0,0,77a29837bdfd97f18040622b1984ee7b87e863b8,0,0,0,0,0,
1,20200923,AL,147153.0,938451.0,NaN,1070903.0,768.0,16698.0,NaN,1669.0,...,1085604,31,94,64268ac422e3a504e9b8401b29b1a739bf4ee1ba,0,0,0,0,0,
2,20200923,AR,77963.0,831301.0,NaN,906947.0,460.0,5098.0,221.0,NaN,...,909264,20,43,052de2d72685a74636cebc28c54f51ef7a5c0bcb,0,0,0,0,0,
3,20200923,AS,0.0,1571.0,NaN,1571.0,NaN,NaN,NaN,NaN,...,1571,0,0,2c46457ee4f802b1f3f14dabbca92ccfbf5670d7,0,0,0,0,0,
4,20200923,AZ,215284.0,1194582.0,NaN,1409866.0,583.0,21925.0,114.0,NaN,...,1409866,27,25,3db97c866c945701ac6812c274b470a388262d73,0,0,0,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11405,20200124,WA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,ca19e6147db94a84ba9cb6aca79e3ea7cb2f6941,0,0,0,0,0,
11406,20200123,MA,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,0,0,0,45e5e53c94482161c8d08f30d01454749e2c59aa,0,0,0,0,0,
11407,20200123,WA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,89e9cc157bfc766d7541dd308afe8ff256bf9440,0,0,0,0,0,
11408,20200122,MA,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,0,0,0,11371837082f36f5c93a58a92e4475138927218e,0,0,0,0,0,


In [48]:
outputPath = os.path.join("..", "Resources", "covidTrackingCurrent.csv")

In [49]:
covidTrackingDF.to_csv(outputPath, index = False)

In [50]:
# Date, state, postive, negative, deathIncrease
# Remove NaN's, Replace with NaN's
# Change full state to abbreviation
# Drop all rows that are not the current date

In [46]:
# Read back in CSV for Cleaning
covidTrackingCSV = pd.read_csv("Resources/covidTrackingCurrent.csv")
covidTrackingCSV

FileNotFoundError: [Errno 2] File Resources/covidTrackingCurrent.csv does not exist: 'Resources/covidTrackingCurrent.csv'

In [38]:
# Pathway to use later to export CSV for cleaning.
us_covid_tracking_data_path = "/resources/us_covid_tracking_data.csv"

# URL for the API call
us_covid_tracking_data = "https://finnhub.io/api/v1/covid19/us"

# Calling API and printing response.
responseFinnhub = requests.get(us_covid_tracking_data).json()
print(json.dumps(responseFinnhub, indent=4, sort_keys=True))

[
    {
        "case": 456262,
        "death": 33090,
        "state": "New York",
        "updated": "2020-09-24 05:00:26"
    },
    {
        "case": 201128,
        "death": 16208,
        "state": "New Jersey",
        "updated": "2020-09-24 05:00:26"
    },
    {
        "case": 793535,
        "death": 15210,
        "state": "California",
        "updated": "2020-09-24 05:00:26"
    },
    {
        "case": 131263,
        "death": 7018,
        "state": "Michigan",
        "updated": "2020-09-24 05:00:26"
    },
    {
        "case": 690499,
        "death": 13618,
        "state": "Florida",
        "updated": "2020-09-24 05:00:26"
    },
    {
        "case": 125867,
        "death": 9328,
        "state": "Massachusetts",
        "updated": "2020-09-24 05:00:26"
    },
    {
        "case": 83215,
        "death": 2151,
        "state": "Washington",
        "updated": "2020-09-24 05:00:26"
    },
    {
        "case": 163869,
        "death": 5407,
        "state": "Loui

In [42]:
responseFinnhubDF = pd.DataFrame(responseFinnhub)
responseFinnhubDF

,state,case,death,updated
0,New York,456262,33090,2020-09-24 05:00:26
1,New Jersey,201128,16208,2020-09-24 05:00:26
2,California,793535,15210,2020-09-24 05:00:26
3,Michigan,131263,7018,2020-09-24 05:00:26
4,Florida,690499,13618,2020-09-24 05:00:26
...,...,...,...,...
57,Wuhan Evacuee,4,0,2020-09-24 05:00:26
58,Northern Mariana Islands,31,2,2020-09-24 05:00:26
59,US Military,63568,94,2020-09-24 05:00:26
60,Federal Bureau of Prisons,16263,124,2020-09-24 05:00:26


In [44]:
outputPath = os.path.join("..", "Resources", "FinnhubCovidCurrent.csv")

In [45]:
responseFinnhubDF.to_csv(outputPath, index = False)

In [ ]:
# Change full state to abbreviation
# Drop rows 55 - 63, Grand Princess
# Make new column with converted date

In [ ]:
# Merge on Date and State